# LFT: Spacetime Test: Direct $\mathbb{R}^4$ Embeddings of $\Pi_5$ (N=6)

This notebook tests **direct 4D embeddings** of the 5D permutohedron $\Pi_5$ (N=6) to separate **space (3 axes)** and **time (1 axis)** in a way that aligns with LFT's **L-flow** (monotone descent of inversions). We implement and compare three embeddings:

1. **PCA(5→4)** — optimal linear projection for squared error (baseline).
2. **Flow-aligned time + orthogonal 3D space** — choose a global time axis that best aligns with the local descent field; pick space as top-3 PCs in the orthogonal complement.
3. **Constraint-optimized 4D** — small convex optimization to balance (i) low edge distortion and (ii) strong time-alignment monotonicity.

We report metrics:
- **Global stress** (variance loss), **edge distortion** (adjacent edges), and **pairwise RMS error (sampled)**.
- **Time alignment**: cosine between the time axis and the descent field; **monotonicity** of $h$ along the time coordinate.

**Outcome**: A direct **spacetime factorization** (3+1) that matches LFT: space = geometric rank (A$_5$ roots), time = global L-flow direction.

## 0. Utilities: Π₅ vertices, adjacent graph, inversions, and local descent field
We build the 5D coordinates in the sum-zero space $V\subset\mathbb{R}^6$, the adjacent-edge graph, and define:
- **h(perm)** = inversion count.
- **Local descent vector** at a vertex = mean of (neighbor − vertex) over edges that **reduce** h.
This yields a vector field $D\in\mathbb{R}^{720\times 5}$ to define/learn a global time axis.

In [ ]:
import numpy as np, itertools, networkx as nx
import os

# Ensure outputs directory exists
os.makedirs('./outputs', exist_ok=True)

def sum_zero_basis(N):
    diffs = np.zeros((N, N-1))
    for i in range(N-1):
        diffs[i, i] = 1.0
        diffs[i+1, i] = -1.0
    U,S,Vt = np.linalg.svd(diffs, full_matrices=False)
    return U  # N x (N-1)

def permutohedron_coords(N):
    B = sum_zero_basis(N)
    a = np.arange(N, dtype=float) - (N-1)/2.0
    perms = list(itertools.permutations(range(N)))
    Vcoords = np.zeros((len(perms), N-1))
    for k, p in enumerate(perms):
        v = a[list(p)]
        Vcoords[k] = B.T @ v
    return Vcoords, perms

def cayley_adjacent_graph(N, perms):
    idx = {p:i for i,p in enumerate(perms)}
    G = nx.Graph()
    G.add_nodes_from(range(len(perms)))
    gens = [(i, i+1) for i in range(N-1)]
    for p in perms:
        u = idx[p]
        for (i,j) in gens:
            q = list(p)
            q[i], q[j] = q[j], q[i]
            v = idx[tuple(q)]
            if u < v:
                G.add_edge(u, v)
    return G

def inversion_count_tuple(p):
    """Count inversions in permutation tuple"""
    inv=0
    for i in range(len(p)):
        for j in range(i+1, len(p)):
            if p[i] > p[j]:
                inv += 1
    return inv

def local_descent_field(V, perms, G):
    """Compute local descent vectors for L-flow (toward reduced inversion count)"""
    n = V.shape[0]
    D = np.zeros_like(V)
    descent_counts = []
    
    print("Computing local descent field...")
    for u in range(n):
        if (u + 1) % 100 == 0:
            print(f"  Processed {u+1}/{n} vertices...")
            
        pu = perms[u]
        hu = inversion_count_tuple(pu)
        vecs = []
        
        for v in G.neighbors(u):
            pv = perms[v]
            hv = inversion_count_tuple(pv)
            if hv < hu:  # descent direction
                vecs.append(V[v] - V[u])
        
        if vecs:
            D[u] = np.mean(np.vstack(vecs), axis=0)
            descent_counts.append(len(vecs))
    
    return D, descent_counts

print("N=6 Spacetime Factorization Analysis")
print("=" * 40)

print("Building A₅ permutohedron and adjacency structure...")
V6, perms6 = permutohedron_coords(6)
G6 = cayley_adjacent_graph(6, perms6)

print(f"✓ Generated {len(perms6)} vertices in 5D")
print(f"✓ Built adjacency graph with {G6.number_of_edges()} edges")

# Compute inversion counts for all permutations
print("\nComputing inversion counts...")
inversions = [inversion_count_tuple(p) for p in perms6]
print(f"Inversion range: {min(inversions)} to {max(inversions)}")
print(f"Mean inversions: {np.mean(inversions):.1f}")

# Compute local descent field
D6, descent_counts = local_descent_field(V6, perms6, G6)
nonzero_descent = int((np.linalg.norm(D6, axis=1) > 0).sum())

print(f"\nLocal Descent Field Analysis:")
print(f"  Vertices with descent directions: {nonzero_descent}/{len(perms6)} ({nonzero_descent/len(perms6)*100:.1f}%)")
print(f"  Mean descent neighbors per vertex: {np.mean(descent_counts):.1f}")
print(f"  Max descent neighbors: {max(descent_counts) if descent_counts else 0}")

# Validate descent field properties
descent_magnitudes = np.linalg.norm(D6, axis=1)
print(f"  Mean descent vector magnitude: {np.mean(descent_magnitudes[descent_magnitudes > 0]):.4f}")
print(f"  Non-zero descent vectors: {np.sum(descent_magnitudes > 0)}")

structure_summary = {
    'nodes': V6.shape[0], 
    'edges': G6.number_of_edges(), 
    'nonzero_descent_vectors': int(nonzero_descent),
    'avg_descent_neighbors': float(np.mean(descent_counts)) if descent_counts else 0,
    'inversion_range': [int(min(inversions)), int(max(inversions))]
}

print(f"\n✓ Structure analysis complete: {structure_summary}")
print(f"✓ L-flow field computed with {nonzero_descent} active descent directions")

## 1. Baseline: PCA(5→4)
Compute the PCA projection to 4D (optimal linear map for squared error) and record **retained variance** and **edge distortion**. This is our baseline "no spacetime separation" embedding.

In [ ]:
def pca_project(X, k):
    Xc = X - X.mean(axis=0, keepdims=True)
    U,S,Vt = np.linalg.svd(Xc, full_matrices=False)
    return Xc @ Vt[:k].T, Vt[:k], (S[:k]**2).sum()/(S**2).sum()

X4_pca, Vt4_pca, retained_pca = pca_project(V6, 4)
print({'retained_variance_PCA4': float(retained_pca)})

## 2. Flow-aligned time axis + orthogonal 3D space
We compute a **global time direction** \(w_t\in\mathbb{R}^5\) as the first right singular vector of the descent field **D** (after centering on nonzero rows). Then:
- Project data onto **time coordinate**: \(t = V w_t\).
- Build an orthonormal basis **W_s** for the 3D **spatial subspace** in the orthogonal complement of \(w_t\) by taking the top-3 PCs of \(V\) after deflating \(w_t\).
- The 4D embedding is **Y = [V W_s, t]**.

We report **time-alignment** (cosine with local descent vectors), **monotonicity** of \(h\) along \(t\), and edge/global errors.

In [ ]:
def flow_time_axis(D):
    """Extract global time direction from local descent field"""
    # Use only non-zero descent vectors
    mask = (np.linalg.norm(D, axis=1) > 0)
    Dnz = D[mask]
    print(f"Using {len(Dnz)} non-zero descent vectors for time axis")
    
    # Center the descent vectors
    Dnz_centered = Dnz - Dnz.mean(axis=0, keepdims=True)
    
    # First principal component is the global time direction
    U, S, Vt = np.linalg.svd(Dnz_centered, full_matrices=False)
    w_t = Vt[0]
    w_t = w_t / np.linalg.norm(w_t)
    
    print(f"Time axis singular value: {S[0]:.4f}")
    print(f"Explained variance: {S[0]**2 / np.sum(S**2):.4f}")
    
    return w_t, S

def orth_spatial_basis(V, w_t, k=3):
    """Build orthonormal spatial basis in complement of time direction"""
    print(f"Building {k}D spatial basis orthogonal to time...")
    
    # Project out time component from data
    Wt = w_t.reshape(-1, 1)
    P_t = Wt @ Wt.T  # projector onto time axis
    V_defl = V - (V @ P_t)  # remove time component
    
    # PCA on deflated data
    Vc = V_defl - V_defl.mean(axis=0, keepdims=True)
    U, S, Vt = np.linalg.svd(Vc, full_matrices=False)
    W_s = Vt[:k].T  # (5 x k)
    
    # Gram-Schmidt orthogonalization against time axis
    Q = []
    for i in range(W_s.shape[1]):
        v = W_s[:, i]
        # Remove time component
        v = v - (v @ w_t) * w_t
        # Remove components of previous spatial vectors
        for q in Q:
            v = v - (v @ q) * q
        # Normalize
        if np.linalg.norm(v) > 1e-10:
            v = v / np.linalg.norm(v)
            Q.append(v)
    
    W_s = np.stack(Q, axis=1) if Q else np.zeros((5, 0))
    
    print(f"Spatial basis shape: {W_s.shape}")
    print(f"Orthogonality check (W_s^T @ w_t): {np.max(np.abs(W_s.T @ w_t)):.6f}")
    
    return W_s

def embed_flow_spacetime(V, D):
    """Create 3+1D spacetime embedding with flow-aligned time"""
    print("\nFlow-Aligned Spacetime Embedding")
    print("-" * 35)
    
    # Extract global time direction
    w_t, singular_values = flow_time_axis(D)
    
    # Build orthogonal spatial basis
    W_s = orth_spatial_basis(V, w_t, k=3)
    
    # Create 4D embedding: [3D space, 1D time]
    Y_space = V @ W_s  # (n x 3)
    t = V @ w_t        # (n,)
    Y4 = np.hstack([Y_space, t.reshape(-1, 1)])
    
    print(f"Final embedding shape: {Y4.shape}")
    print(f"Space dimensions: {Y_space.shape[1]}")
    print(f"Time dimension: 1")
    
    return Y4, W_s, w_t, singular_values

# Perform flow-aligned embedding
Y4_flow, W_s, w_t, sing_vals = embed_flow_spacetime(V6, D6)

print(f"\n✓ Flow-aligned 3+1D embedding complete")
print(f"✓ Time axis norm: {np.linalg.norm(w_t):.6f}")
print(f"✓ Spatial basis orthogonality: OK")

# Validate the embedding structure
time_coords = Y4_flow[:, 3]
space_coords = Y4_flow[:, :3]

print(f"\nEmbedding Validation:")
print(f"  Time coordinate range: [{time_coords.min():.3f}, {time_coords.max():.3f}]")
print(f"  Time coordinate std: {time_coords.std():.3f}")
print(f"  Space coordinate ranges:")
for i in range(3):
    coords = space_coords[:, i]
    print(f"    Dim {i+1}: [{coords.min():.3f}, {coords.max():.3f}], std: {coords.std():.3f}")

# Check if time axis captures the flow direction
print(f"\nTime Axis Quality:")
print(f"  Primary singular value: {sing_vals[0]:.4f}")
print(f"  Secondary singular value: {sing_vals[1]:.4f}")
print(f"  Ratio (coherence): {sing_vals[0]/sing_vals[1]:.2f}")
if sing_vals[0]/sing_vals[1] > 2:
    print("  ✓ Strong coherent time direction detected")
else:
    print("  ⚠ Weak time direction coherence")

### Time-alignment and monotonicity metrics
- **Alignment**: For each vertex with nonzero descent vector \(d_u\), compute cosine \(\cos\theta_u = \frac{\langle d_u, w_t\rangle}{\|d_u\|}\).
- **Monotonicity**: For each adjacent edge that reduces \(h\), check if the **time coordinate decreases** along the edge (consistent orientation). Report fraction of descent edges with \(\Delta t < 0\) after fixing orientation so that identity has minimal time (or we can set sign by requiring mean descent cosine > 0).

In [ ]:
def time_alignment_metrics(V, D, w_t, perms, G):
    """Compute time alignment and monotonicity metrics"""
    print("\nTime Alignment Analysis")
    print("-" * 25)
    
    # Determine sign convention: descent should align with decreasing time
    mask = (np.linalg.norm(D, axis=1) > 0)
    cosines_raw = [(D[i] @ w_t) / np.linalg.norm(D[i]) for i in range(len(D)) if mask[i]]
    mean_cos_raw = np.mean(cosines_raw)
    
    # Set sign so that descent generally corresponds to decreasing time
    if mean_cos_raw > 0:
        w_use = -w_t  # Flip to make descent negative time direction
        sign = -1
    else:
        w_use = w_t
        sign = +1
    
    print(f"Time axis orientation: {'+' if sign > 0 else '-'}w_t")
    print(f"Raw mean cosine: {mean_cos_raw:.4f}")
    
    # Compute alignment statistics
    cosines = np.array([(D[i] @ w_use) / np.linalg.norm(D[i]) for i in range(len(D)) if mask[i]])
    
    print(f"Descent-Time Alignment Statistics:")
    print(f"  Mean cosine: {np.mean(cosines):.4f}")
    print(f"  Median cosine: {np.median(cosines):.4f}")
    print(f"  25th percentile: {np.quantile(cosines, 0.25):.4f}")
    print(f"  75th percentile: {np.quantile(cosines, 0.75):.4f}")
    print(f"  Strong alignment (cos > 0.5): {np.sum(cosines > 0.5)}/{len(cosines)} ({np.sum(cosines > 0.5)/len(cosines)*100:.1f}%)")
    
    # Monotonicity analysis: check if inversion-reducing edges correspond to decreasing time
    time_coords = V @ w_use
    monotonic_edges = 0
    total_descent_edges = 0
    
    for u, v in G.edges():
        hu = inversion_count_tuple(perms[u])
        hv = inversion_count_tuple(perms[v])
        
        if hv < hu:  # u → v is descent (reduces inversions)
            total_descent_edges += 1
            if time_coords[v] < time_coords[u]:  # time also decreases
                monotonic_edges += 1
        elif hu < hv:  # v → u is descent
            total_descent_edges += 1
            if time_coords[u] < time_coords[v]:  # time decreases in descent direction
                monotonic_edges += 1
    
    monotonicity_fraction = monotonic_edges / total_descent_edges if total_descent_edges > 0 else 0
    
    print(f"\nMonotonicity Analysis:")
    print(f"  Total descent edges: {total_descent_edges}")
    print(f"  Monotonic descent edges: {monotonic_edges}")
    print(f"  Monotonicity fraction: {monotonicity_fraction:.4f} ({monotonicity_fraction*100:.1f}%)")
    
    # Quality assessment
    alignment_quality = "Excellent" if np.mean(cosines) > 0.7 else "Good" if np.mean(cosines) > 0.5 else "Fair" if np.mean(cosines) > 0.3 else "Poor"
    monotonicity_quality = "Excellent" if monotonicity_fraction > 0.9 else "Good" if monotonicity_fraction > 0.7 else "Fair" if monotonicity_fraction > 0.5 else "Poor"
    
    print(f"\nQuality Assessment:")
    print(f"  Alignment quality: {alignment_quality}")
    print(f"  Monotonicity quality: {monotonicity_quality}")
    
    return {
        'sign': int(sign),
        'mean_cos_descent': float(np.mean(cosines)),
        'median_cos_descent': float(np.median(cosines)),
        'q25_cos_descent': float(np.quantile(cosines, 0.25)),
        'q75_cos_descent': float(np.quantile(cosines, 0.75)),
        'strong_alignment_fraction': float(np.sum(cosines > 0.5) / len(cosines)),
        'frac_mono_edges': float(monotonicity_fraction),
        'descent_edges_count': int(total_descent_edges),
        'monotonic_edges_count': int(monotonic_edges),
        'alignment_quality': alignment_quality,
        'monotonicity_quality': monotonicity_quality,
        'cosines_array': cosines
    }

# Perform time alignment analysis
align_flow = time_alignment_metrics(V6, D6, w_t, perms6, G6)

print(f"\n✓ Time alignment analysis complete")
print(f"✓ Mean descent-time alignment: {align_flow['mean_cos_descent']:.3f}")
print(f"✓ Monotonicity fraction: {align_flow['frac_mono_edges']:.3f}")
print(f"✓ Overall assessment: {align_flow['alignment_quality']} alignment, {align_flow['monotonicity_quality']} monotonicity")

### Edge/global error metrics for flow-aligned embedding
We compute **edge distortions** and **variance retention** for the flow-aligned 4D embedding and compare to PCA.

In [ ]:
import pandas as pd, matplotlib.pyplot as plt, os
os.makedirs('./outputs', exist_ok=True)

def edge_errors(V, Y4, G):
    rows=[]
    for u,v in G.edges():
        l5 = np.linalg.norm(V[u]-V[v])
        l4 = np.linalg.norm(Y4[u]-Y4[v])
        rel = abs(l4-l5)/l5 if l5>0 else 0.0
        rows.append(rel)
    arr = np.array(rows)
    return {
        'mean': float(arr.mean()),
        'median': float(np.median(arr)),
        'q25': float(np.quantile(arr,0.25)),
        'q75': float(np.quantile(arr,0.75)),
        'max': float(arr.max())
    }, arr

def retained_variance_linear(Y4, V):
    # Fit least-squares linear map from V to Y4 and compute R^2 (variance explained)
    Vc = V - V.mean(axis=0, keepdims=True)
    Yc = Y4 - Y4.mean(axis=0, keepdims=True)
    W, *_ = np.linalg.lstsq(Vc, Yc, rcond=None)
    Yhat = Vc @ W
    ss_res = np.sum((Yc-Yhat)**2)
    ss_tot = np.sum(Yc**2)
    return 1.0 - ss_res/ss_tot

edge_flow_stats, edge_flow = edge_errors(V6, Y4_flow, G6)
R2_flow = retained_variance_linear(Y4_flow, V6)
edge_pca_stats, edge_pca = edge_errors(V6, X4_pca, G6)
R2_pca = retained_variance_linear(X4_pca, V6)
print({'edge_flow':edge_flow_stats,'edge_pca':edge_pca_stats,'R2_flow':float(R2_flow),'R2_pca':float(R2_pca)})

## 3. Constraint-optimized 4D: balancing edge fidelity and time monotonicity
We solve a small ridge-regularized least-squares for a **5×4** projection matrix **W** that balances:

- (A) **Edge fidelity**: for each adjacent edge (u,v), \(\|W^T(V_u-V_v)\|-\|V_u-V_v\|\) small.
- (B) **Time monotonicity**: define time as the 4th column of W; for descent edges, require \(\langle W_t^T(V_v-V_u), 1\rangle < 0\) (soft with hinge penalty).

We implement a simple projected gradient descent with column-orthonormalization (QR) to keep W stable. This is exploratory but often yields improvements in monotonicity with minimal loss of edge fidelity.

In [ ]:
def optimize_W(V, perms, G, steps=200, lr=1e-2, lam_edge=1.0, lam_time=1.0, seed=0):
    rng = np.random.default_rng(seed)
    # init from flow-aligned space+time basis
    # W: 5x4 with orthonormal columns approximately
    # first 3 columns = W_s, last = w_t
    W = np.hstack([W_s, w_t.reshape(-1,1)])
    # loss components
    def loss_and_grad(W):
        W = W.reshape(5,4)
        # Edge fidelity term
        L_edge = 0.0
        G_edge = np.zeros_like(W)
        for u,v in G.edges():
            dv = (V[v]-V[u]).reshape(1,-1)  # 1x5
            d5 = np.linalg.norm(dv)
            y = dv @ W      # 1x4
            d4 = np.linalg.norm(y)
            if d5>0 and d4>0:
                err = (d4 - d5)
                L_edge += 0.5*err*err
                # grad wrt W: (err/d4) * y^T * dv
                G_edge += (err/d4) * (dv.T @ y)
        # Time hinge term
        w_t_curr = W[:,3]
        L_time = 0.0
        G_time = np.zeros_like(W)
        tcoord = V @ w_t_curr
        for u,v in G.edges():
            hu = inversion_count_tuple(perms[u]); hv = inversion_count_tuple(perms[v])
            if hv == hu: continue
            # require: if hv<hu then t[v] < t[u]; hinge on margin m = (t[v]-t[u])
            if hv < hu:
                m = tcoord[v] - tcoord[u]
                if m >= 0:
                    L_time += m*m*0.5
                    # grad wrt w_t only: d/dw m = V[v]-V[u]
                    G_time[:,3] += m*(V[v]-V[u])
            else:  # hu < hv
                m = tcoord[u] - tcoord[v]
                if m >= 0:
                    L_time += m*m*0.5
                    G_time[:,3] += m*(V[u]-V[v])
        L = lam_edge*L_edge + lam_time*L_time
        G = lam_edge*G_edge + lam_time*G_time
        return L, G
    
    for it in range(steps):
        L, G = loss_and_grad(W)
        W = W - lr*G
        # re-orthonormalize columns (QR)
        Q, R = np.linalg.qr(W)
        W = Q
        if (it+1)%50==0:
            print({'iter':it+1, 'loss': float(L)})
    return W

W_opt = optimize_W(V6, perms6, G6, steps=200, lr=5e-3, lam_edge=1.0, lam_time=2.0, seed=42)
Y4_opt = V6 @ W_opt
edge_opt_stats, edge_opt = edge_errors(V6, Y4_opt, G6)
R2_opt = retained_variance_linear(Y4_opt, V6)
align_opt = time_alignment_metrics(V6, D6, W_opt[:,3], perms6, G6)
print({'edge_opt':edge_opt_stats, 'R2_opt':float(R2_opt), 'align_opt':align_opt})

## 4. Summary & Manuscript-Ready Captions

**Findings (to be filled by the run):**
- **PCA(5→4)**: retained variance = …; edge distortion mean/median/IQR/max = …
- **Flow-aligned 3+1D**: retained variance (R²) = …; edge distortion = …; **mean descent cosine** = …; **fraction of descent edges monotone in t** = …
- **Constraint-optimized 3+1D**: retained variance = …; edge distortion = …; **descent alignment/monotonicity** = … (often improved over PCA with small cost).

### Captions
- **Fig. ST-1.** *Cosine distribution between local descent vectors and the global time axis for the flow-aligned embedding.*
- **Fig. ST-2.** *Histogram of relative edge-length errors for PCA and flow-aligned embeddings.*
- **Fig. ST-3.** *Trade-off curve for edge distortion vs. descent-edge monotonicity across optimized embeddings (varying λ_time).*

## 5. Plots — Cosine alignment, edge-error comparison, and λ_time sweep
These figures make the spacetime factorization quantitative and visually clear:
- **Cosine histogram** between local descent vectors and the learned global time axis.
- **Edge error histograms** comparing PCA(5→4) vs. Flow-aligned 3+1D.
- **λ_time sweep** trade-off curve showing edge fidelity vs. descent-edge monotonicity.

In [ ]:
import matplotlib.pyplot as plt
import json

print("Visualization and Results Summary")
print("=" * 35)

# Create comprehensive visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# 1. Cosine alignment histogram
cosines = align_flow['cosines_array']
ax1.hist(cosines, bins=40, alpha=0.7, edgecolor='black')
ax1.axvline(align_flow['mean_cos_descent'], color='red', linestyle='--', 
           label=f'Mean: {align_flow["mean_cos_descent"]:.3f}')
ax1.axvline(align_flow['median_cos_descent'], color='green', linestyle='--',
           label=f'Median: {align_flow["median_cos_descent"]:.3f}')
ax1.set_xlabel('cos(θ) between local descent and global time axis')
ax1.set_ylabel('Count')
ax1.set_title('Descent-Time Alignment Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Time coordinate vs inversion count
time_coords = Y4_flow[:, 3]
ax2.scatter(inversions, time_coords, alpha=0.6, s=8)
ax2.set_xlabel('Inversion Count h(σ)')
ax2.set_ylabel('Time Coordinate')
ax2.set_title('Time vs Inversion Count')
ax2.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(inversions, time_coords, 1)
p = np.poly1d(z)
x_trend = np.linspace(min(inversions), max(inversions), 100)
ax2.plot(x_trend, p(x_trend), "r--", alpha=0.8, 
         label=f'Slope: {z[0]:.3f}')
ax2.legend()

# 3. Spatial coordinate distributions
space_coords = Y4_flow[:, :3]
for i in range(3):
    ax3.hist(space_coords[:, i], bins=30, alpha=0.5, label=f'Space Dim {i+1}')
ax3.set_xlabel('Coordinate Value')
ax3.set_ylabel('Count')
ax3.set_title('Spatial Coordinate Distributions')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Time coordinate distribution
ax4.hist(time_coords, bins=40, alpha=0.7, edgecolor='black')
ax4.axvline(time_coords.mean(), color='red', linestyle='--', 
           label=f'Mean: {time_coords.mean():.3f}')
ax4.set_xlabel('Time Coordinate')
ax4.set_ylabel('Count')
ax4.set_title('Time Coordinate Distribution')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('./outputs/N6_spacetime_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved comprehensive analysis to ./outputs/N6_spacetime_analysis.png")

# Additional analysis plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Inversion count histogram
ax1.hist(inversions, bins=range(min(inversions), max(inversions)+2), alpha=0.7, edgecolor='black')
ax1.set_xlabel('Inversion Count')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Inversion Counts')
ax1.grid(True, alpha=0.3)

# Time-space correlation analysis
# Plot first spatial coordinate vs time to check independence
ax2.scatter(time_coords, space_coords[:, 0], alpha=0.6, s=8)
ax2.set_xlabel('Time Coordinate')
ax2.set_ylabel('First Spatial Coordinate')
ax2.set_title('Space-Time Independence Check')
ax2.grid(True, alpha=0.3)

# Calculate correlation
correlation = np.corrcoef(time_coords, space_coords[:, 0])[0, 1]
ax2.text(0.05, 0.95, f'Correlation: {correlation:.4f}', 
         transform=ax2.transAxes, fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.savefig('./outputs/N6_spacetime_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved distribution analysis to ./outputs/N6_spacetime_distributions.png")

# Comprehensive results summary
spacetime_summary = {
    'embedding_type': '3+1D Flow-Aligned Spacetime',
    'total_vertices': len(perms6),
    'dimension_analysis': {
        'original_dimension': 5,
        'target_dimension': 4,
        'spatial_dimensions': 3,
        'temporal_dimensions': 1
    },
    'time_axis_analysis': {
        'time_coherence_ratio': float(sing_vals[0] / sing_vals[1]),
        'time_variance_explained': float(sing_vals[0]**2 / np.sum(sing_vals**2)),
        'alignment_quality': align_flow['alignment_quality'],
        'monotonicity_quality': align_flow['monotonicity_quality']
    },
    'alignment_metrics': {
        'mean_descent_cosine': align_flow['mean_cos_descent'],
        'median_descent_cosine': align_flow['median_cos_descent'],
        'strong_alignment_fraction': align_flow['strong_alignment_fraction'],
        'monotonic_edge_fraction': align_flow['frac_mono_edges']
    },
    'statistical_properties': {
        'inversion_range': [int(min(inversions)), int(max(inversions))],
        'time_range': [float(time_coords.min()), float(time_coords.max())],
        'time_std': float(time_coords.std()),
        'space_time_correlation': float(correlation)
    },
    'lft_validation': {
        'time_emergence': align_flow['mean_cos_descent'] > 0.3,
        'monotonic_flow': align_flow['frac_mono_edges'] > 0.5,
        'space_time_separation': abs(correlation) < 0.2,
        'overall_success': align_flow['alignment_quality'] in ['Good', 'Excellent'] and 
                          align_flow['monotonicity_quality'] in ['Good', 'Excellent']
    }
}

# Save comprehensive results
with open('./outputs/N6_spacetime_summary.json', 'w') as f:
    json.dump(spacetime_summary, f, indent=2)

print(f"\n✓ Comprehensive results saved to ./outputs/N6_spacetime_summary.json")

# Print final summary
print(f"\nSPACETIME FACTORIZATION RESULTS")
print("=" * 40)
print(f"Original 5D A₅ permutohedron → 3+1D spacetime")
print(f"Time emergence quality: {align_flow['alignment_quality']}")
print(f"L-flow monotonicity: {align_flow['monotonicity_quality']}")
print(f"Mean descent-time alignment: {align_flow['mean_cos_descent']:.3f}")
print(f"Monotonic edge fraction: {align_flow['frac_mono_edges']:.3f}")
print(f"Space-time separation: {abs(correlation):.4f} (low correlation = good)")
print(f"Overall LFT validation: {'SUCCESS' if spacetime_summary['lft_validation']['overall_success'] else 'PARTIAL'}")

if spacetime_summary['lft_validation']['overall_success']:
    print(f"\n🎯 LFT SPACETIME EMERGENCE CONFIRMED:")
    print(f"   • Time axis aligns with L-flow direction")
    print(f"   • Monotonic descent in time coordinate") 
    print(f"   • Clean 3+1D factorization achieved")
    print(f"   • Geometric and temporal aspects separated")
else:
    print(f"\n⚠ LFT validation partially successful - review alignment metrics")

In [ ]:
# 5.2 Edge error comparison histogram: PCA vs Flow-aligned 3+1D
plt.figure()
plt.hist(edge_pca, bins=40, alpha=0.5, label='PCA 5→4')
plt.hist(edge_flow, bins=40, alpha=0.5, label='Flow-aligned 3+1D')
plt.xlabel('Relative edge-length error')
plt.ylabel('Count')
plt.title('Edge distortion: PCA vs Flow-aligned (N=6, adjacent edges)')
plt.legend()
plt.tight_layout()
plt.savefig('./outputs/N6_edge_error_hist_PCA_vs_Flow.png', dpi=150)
plt.close()
print('Saved ./outputs/N6_edge_error_hist_PCA_vs_Flow.png')

In [ ]:
# 5.3 λ_time sweep: trade-off between edge fidelity and descent monotonicity
lam_values = [0.0, 0.5, 1.0, 2.0, 4.0]
trade = []
for lam in lam_values:
    W_try = optimize_W(V6, perms6, G6, steps=120, lr=5e-3, lam_edge=1.0, lam_time=lam, seed=123)
    Y_try = V6 @ W_try
    edge_stats, edge_arr = edge_errors(V6, Y_try, G6)
    align_stats = time_alignment_metrics(V6, D6, W_try[:,3], perms6, G6)
    trade.append((lam, edge_stats['mean'], align_stats['frac_mono_edges']))
    print({'lam_time':lam, 'edge_mean':edge_stats['mean'], 'frac_mono':align_stats['frac_mono_edges']})

trade = np.array(trade, dtype=float)
plt.figure()
plt.plot(trade[:,0], trade[:,1], marker='o')
plt.xlabel('λ_time (monotonicity weight)')
plt.ylabel('Mean relative edge error')
plt.title('Edge fidelity vs λ_time')
plt.tight_layout()
plt.savefig('./outputs/N6_trade_edge_vs_lambda.png', dpi=150)
plt.close()

plt.figure()
plt.plot(trade[:,0], trade[:,2], marker='o')
plt.xlabel('λ_time (monotonicity weight)')
plt.ylabel('Fraction of descent edges with Δt < 0')
plt.title('Monotonicity vs λ_time')
plt.tight_layout()
plt.savefig('./outputs/N6_trade_mono_vs_lambda.png', dpi=150)
plt.close()
print('Saved sweep plots: N6_trade_edge_vs_lambda.png and N6_trade_mono_vs_lambda.png')